In [ ]:
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf
import numpy as np
from skimage.io import imread, imshow
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [17]:
image = img_to_array(load_img('./input_images/10000144A.jpg'))
image = np.array(image, dtype=float)

In [18]:
image

array([[[ 97.,  91.,  65.],
        [147., 141., 115.],
        [163., 157., 131.],
        ...,
        [197., 196., 191.],
        [187., 184., 167.],
        [186., 183., 166.]],

       [[133., 131., 108.],
        [183., 181., 158.],
        [200., 198., 177.],
        ...,
        [221., 220., 215.],
        [217., 213., 201.],
        [216., 212., 200.]],

       [[157., 155., 142.],
        [209., 207., 194.],
        [226., 224., 211.],
        ...,
        [230., 229., 225.],
        [222., 219., 210.],
        [220., 217., 208.]],

       ...,

       [[ 36.,  43.,  49.],
        [ 39.,  50.,  52.],
        [ 46.,  58.,  54.],
        ...,
        [221., 217., 208.],
        [229., 222., 206.],
        [213., 206., 190.]],

       [[ 28.,  45.,  55.],
        [ 33.,  53.,  60.],
        [ 29.,  50.,  51.],
        ...,
        [220., 216., 207.],
        [227., 220., 202.],
        [211., 204., 186.]],

       [[ 17.,  43.,  56.],
        [ 28.,  58.,  68.],
        [ 20.,  

In [1]:
def print_image_summary(image, labels):
    
    print('--------------')
    print('Image Details:')
    print('--------------')
    print(f'Image dimensions: {image.shape}')
    print('Channels:')
    
    if len(labels) == 1:
        image = image[..., np.newaxis]
        
    for i, lab in enumerate(labels):
        min_val = np.min(image[:,:,i])
        max_val = np.max(image[:,:,i])
        print(f'{lab} : min={min_val:.4f}, max={max_val:.4f}')

In [2]:
image_gs = imread('./input_images/10000144A.jpg', as_gray=True)
fig, ax = plt.subplots(figsize=(9, 16))
imshow(image_gs, ax=ax)
ax.set_title('Grayscale image')
ax.axis('off');

NameError: name 'imread' is not defined